In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [2]:

# Data preparation function for MINUTE data
def prepare_minute_data(ticker, sequence_length=60):
    """
    Prepare minute stock data for LSTM
    sequence_length: number of minutes to look back 
    - 60 = 1 hour
    - 390 = 1 trading day (9:15 AM - 3:30 PM)
    """
    # Load minute data
    df = pd.read_csv(f'Stock Data/{ticker}_candles.csv')
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    
    print(f"Total data points: {len(df)}")
    print(f"Date range: {df.index.min()} to {df.index.max()}")
    
    # Select features (OHLCV)
    features = ['open', 'high', 'low', 'close', 'volume']
    data = df[features].values
    
    # Split data FIRST (before scaling)
    split_idx = int(len(data) * 0.8)
    train_data = data[:split_idx]
    test_data = data[split_idx:]
    
    # Fit scaler on TRAIN data only
    scaler = MinMaxScaler()
    train_normalized = scaler.fit_transform(train_data)
    test_normalized = scaler.transform(test_data)
    
    # Create sequences for train
    X_train, y_train = [], []
    for i in range(len(train_normalized) - sequence_length):
        X_train.append(train_normalized[i:i+sequence_length])
        y_train.append(train_normalized[i+sequence_length, 3])  # Next minute's close
    
    # Create sequences for test
    X_test, y_test = [], []
    for i in range(len(test_normalized) - sequence_length):
        X_test.append(test_normalized[i:i+sequence_length])
        y_test.append(test_normalized[i+sequence_length, 3])
    
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    
    print(f"\nSequences created:")
    print(f"X_train shape: {X_train.shape}")
    print(f"X_test shape: {X_test.shape}")
    
    # Convert to PyTorch tensors
    X_train = torch.FloatTensor(X_train)
    X_test = torch.FloatTensor(X_test)
    y_train = torch.FloatTensor(y_train)
    y_test = torch.FloatTensor(y_test)
    
    return X_train, X_test, y_train, y_test, scaler

# Test data preparation - try with 60-minute lookback first
ticker = "RELIANCE"
X_train, X_test, y_train, y_test, scaler = prepare_minute_data(ticker, sequence_length=60)

Total data points: 604670
Date range: 2017-07-03 09:15:00 to 2024-01-12 15:29:00

Sequences created:
X_train shape: (483676, 60, 5)
X_test shape: (120874, 60, 5)


In [3]:
# Same LSTM Model (reuse from hourly)
class StockLSTM(nn.Module):
    def __init__(self, input_size=5, hidden_size=64, num_layers=2, dropout=0.2):
        super(StockLSTM, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # LSTM layers
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        # Fully connected layers
        self.fc1 = nn.Linear(hidden_size, 32)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(32, 1)
        
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        last_output = lstm_out[:, -1, :]
        out = self.fc1(last_output)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)
        return out.squeeze()

# Initialize model
model = StockLSTM(input_size=5, hidden_size=64, num_layers=2).to(device)
print(model)
print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters())}")

StockLSTM(
  (lstm): LSTM(5, 64, num_layers=2, batch_first=True, dropout=0.2)
  (fc1): Linear(in_features=64, out_features=32, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=32, out_features=1, bias=True)
)

Total parameters: 53569


In [5]:
# Training function with BATCHING (essential for minute data!)
def train_model_batched(model, X_train, y_train, X_test, y_test, 
                        epochs=50, lr=0.0001, batch_size=512):
    """
    Training with mini-batches for efficient processing of large minute-level data
    """
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    # Create data loaders
    train_dataset = TensorDataset(X_train, y_train)
    test_dataset = TensorDataset(X_test, y_test)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    train_losses = []
    test_losses = []
    
    for epoch in range(epochs):
        # Training
        model.train()
        train_loss = 0.0
        
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            
            optimizer.zero_grad()
            predictions = model(batch_X)
            loss = criterion(predictions, batch_y)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * batch_X.size(0)
        
        train_loss /= len(train_loader.dataset)
        
        # Evaluation
        model.eval()
        test_loss = 0.0
        
        with torch.no_grad():
            for batch_X, batch_y in test_loader:
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                predictions = model(batch_X)
                loss = criterion(predictions, batch_y)
                test_loss += loss.item() * batch_X.size(0)
        
        test_loss /= len(test_loader.dataset)
        
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        
        if (epoch + 1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{epochs}] - Train Loss: {train_loss:.6f}, Test Loss: {test_loss:.6f}")
    
    return train_losses, test_losses

# Train the model
print("Starting training...")
train_losses, test_losses = train_model_batched(
    model, X_train, y_train, X_test, y_test, 
    epochs=50, batch_size=512
)

Starting training...
Epoch [10/50] - Train Loss: 0.079129, Test Loss: 0.117801
Epoch [20/50] - Train Loss: 0.078537, Test Loss: 0.121974


KeyboardInterrupt: 

In [ ]:
# Plot training history
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.title(f'{ticker} (Minute Data) - LSTM Training History')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Evaluation and visualization
model.eval()
predictions_list = []
actuals_list = []

test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=512, shuffle=False)

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X = batch_X.to(device)
        batch_pred = model(batch_X).cpu().numpy()
        predictions_list.extend(batch_pred)
        actuals_list.extend(batch_y.numpy())

predictions = np.array(predictions_list)
actuals = np.array(actuals_list)

# Denormalize predictions to actual prices
dummy = np.zeros((len(predictions), 5))
dummy[:, 3] = predictions
predictions_actual = scaler.inverse_transform(dummy)[:, 3]

dummy[:, 3] = actuals
actuals_actual = scaler.inverse_transform(dummy)[:, 3]

# Calculate metrics
mae = np.mean(np.abs(predictions_actual - actuals_actual))
rmse = np.sqrt(np.mean((predictions_actual - actuals_actual)**2))
mape = np.mean(np.abs((actuals_actual - predictions_actual) / actuals_actual)) * 100

print(f"\n{ticker} Minute-Level Model Performance:")
print(f"MAE: ₹{mae:.2f}")
print(f"RMSE: ₹{rmse:.2f}")
print(f"MAPE: {mape:.2f}%")

# Plot predictions vs actuals (sample for readability)
sample_size = min(2000, len(predictions_actual))  # Plot last 2000 minutes
plt.figure(figsize=(14, 6))
plt.plot(actuals_actual[-sample_size:], label='Actual Price', alpha=0.7, linewidth=1)
plt.plot(predictions_actual[-sample_size:], label='Predicted Price', alpha=0.7, linewidth=1)
plt.xlabel('Time Steps (Minutes)')
plt.ylabel('Stock Price (₹)')
plt.title(f'{ticker} - Minute-Level LSTM Predictions vs Actual (Last {sample_size} minutes)')
plt.legend()
plt.grid(True)
plt.show()